In [85]:
import pandas as pd
import os
import json
import requests
from pprint import pprint
import numpy as np

In [3]:
def load_api_keys(file_path):
    """
    Load API keys from a JSON file.
    """
    with open(file_path, 'r') as file:
        keys = json.load(file)
    return keys

In [4]:
keys = load_api_keys('../data/api_keys.json')

In [5]:
def yelp_search(api_key, lat, lon):
    """
    Perform a Yelp search near the given latitude and longitude.
    """
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {'Authorization': f'Bearer {api_key}'}
    params = {
        'term': 'restaurant',
        'latitude': lat,
        'longitude': lon,
        'radius': 1609,  # Convert back to meters for Yelp's radius parameter
        'limit': 50  # Yelp API allows up to 50 results per request
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        print(f"Error: Unable to fetch data from Yelp API. Status Code: {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame if there's an error
    

    data = response.json()

    # Create DataFrame from Yelp API response
    df = pd.DataFrame([{
        'Name': business['name'],
        'Business ID': business.get('id'),
        'Address': ', '.join(business['location'].get('display_address', [])),
        'Rating': business.get('rating'),
        'Total Ratings': business.get('review_count'),
        'Price': business.get('price', None),  # Price may not be available for all businesses
        'Cuisine': business.get('categories')
    } for business in data.get('businesses', [])])
    
    return df

In [6]:
df_points = pd.read_csv('../data/nola.csv', header=None, names=['Latitude', 'Longitude'],skiprows=1)

# Initialize an empty DataFrame to store Yelp results


In [7]:
df_points

,Latitude,Longitude
0,29.956970,-90.112957
1,30.011266,-90.077282
2,29.914587,-90.018549
3,29.975803,-90.046806
4,29.962644,-90.097513
...,...,...
102,29.909259,-90.003386
103,30.001888,-90.003923
104,30.018291,-90.036028
105,29.951151,-90.009855


In [59]:
combined_df = pd.DataFrame({
    'Name': ['existing restaurant'],
    'Business ID': ['id1'],
    'Address': ['100 Existing St'],
    'Rating': [0],
    'Total Ratings': [0],
    'Price': ['$'],
    'Cuisine': [None]
})

In [60]:
for i, (index, row) in enumerate(df_points.iterrows()):
    if i >= 5:  # Stop after 5 iterations
        break    
    
    latitude, longitude = row['Latitude'], row['Longitude']

    print("hey",latitude,longitude)
    # Search Yelp using the latitude and longitude
    yelp_df = yelp_search(keys['yelp_api_key'], latitude, longitude)
    yelp_df['Name'] = yelp_df['Name'].str.lower()

    print(len(yelp_df))
    combined_df = pd.concat([combined_df, yelp_df[~yelp_df['Name'].isin(combined_df['Name'])]], ignore_index=True)

    i+=1


hey 29.956969553435577 -90.1129573456206
50
hey 30.011266383314485 -90.07728191476632
22
hey 29.91458695905421 -90.01854855648119
50
hey 29.975802650965527 -90.04680638143522
50
hey 29.96264442348628 -90.09751262655034
50


In [63]:
combined_df.head(n=20)

,Name,Business ID,Address,Rating,Total Ratings,Price,Cuisine
0,existing restaurant,id1,100 Existing St,0.0,0,$,None
1,ye olde college inn,yY3kq6_2m1FyY9MgaiHgoQ,"3000 S Carrollton Ave, New Orleans, LA 70118",4.2,272,$$,"[{'alias': 'southern', 'title': 'Southern'}, {..."
2,nice guys nola,AZJpX5slpbZUTU9INbwu0g,"7910 Earhart Blvd, New Orleans, LA 70125",3.2,459,$$,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':..."
3,boucherie,rsMbMeW7IbXxPFh4WwbddQ,"8115 Jeannette St, New Orleans, LA 70118",4.3,627,$$,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia..."
4,luca eats,854d-ziK_hQFmbNdvF3ZsA,"7329 Cohn St, New Orleans, LA 70118",4.5,200,$,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias..."
5,que pasta nola,zznZqH9CiAznbkV6fXyHWA,"500 S Telemachus St, New Orleans, LA 70119",4.9,15,None,"[{'alias': 'popuprestaurants', 'title': 'Pop-U..."
6,mikimoto,5rScKeYVjFk8VahsvJv9QA,"3301 S Carrollton Ave, New Orleans, LA 70118",3.9,469,$$,"[{'alias': 'japanese', 'title': 'Japanese'}, {..."
7,gloria's restaurant,QyO7tZ9ECNq162V53PUzmA,"3101 Broadway St, New Orleans, LA 70125",5.0,2,None,"[{'alias': 'breakfast_brunch', 'title': 'Break..."
8,el pavo real,MuJBELBk9VXJjH0tryDh1w,"4401 S Broad Ave, New Orleans, LA 70125",4.2,232,$$,"[{'alias': 'mexican', 'title': 'Mexican'}]"
9,bspot nola,n3OesNK3bdBoLT7XefiOuQ,"3150 Calhoun St, New Orleans, LA 70125",4.1,106,None,"[{'alias': 'soulfood', 'title': 'Soul Food'}, ..."


In [29]:
import pymysql

import mysql.connector

In [64]:
import pymysql

# Database connection details
host = "vibez-database1-instance-1.c74gs48gc437.us-east-1.rds.amazonaws.com"
port = 3306
dbname = "restaurants"
user = "admin"
password = "Banker2b!"


    # Establish a connection to the database
conn = pymysql.connect(host=host, user=user, port=port, password=password, database=dbname)

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Execute the SQL query to get the list of databases
cursor.execute("SHOW DATABASES;")

# Fetch all the results
databases = cursor.fetchall()

# Print out the databases
print("Databases:")
for db in databases:
    print(db[0])



Databases:
information_schema
mysql
performance_schema
restaurants
sys


In [65]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS restaurant_info (
        name VARCHAR(255),
        business_id VARCHAR(255),
        address VARCHAR(255),
        rating FLOAT,
        total_ratings INT,
        price VARCHAR(10),
        cuisine TEXT
    );
    """
cursor.execute(create_table_query)
conn.commit()
print("Table 'restaurant_info' created successfully.")


Table 'restaurant_info' created successfully.


In [96]:
combined_df

,Name,Business ID,Address,Rating,Total Ratings,Price,Cuisine
0,existing restaurant,id1,100 Existing St,0.0,0,$,None
1,ye olde college inn,yY3kq6_2m1FyY9MgaiHgoQ,"3000 S Carrollton Ave, New Orleans, LA 70118",4.2,272,$$,"[{""alias"": ""southern"", ""title"": ""Southern""}, {..."
2,nice guys nola,AZJpX5slpbZUTU9INbwu0g,"7910 Earhart Blvd, New Orleans, LA 70125",3.2,459,$$,"[{""alias"": ""bars"", ""title"": ""Bars""}, {""alias"":..."
3,boucherie,rsMbMeW7IbXxPFh4WwbddQ,"8115 Jeannette St, New Orleans, LA 70118",4.3,627,$$,"[{""alias"": ""bbq"", ""title"": ""Barbeque""}, {""alia..."
4,luca eats,854d-ziK_hQFmbNdvF3ZsA,"7329 Cohn St, New Orleans, LA 70118",4.5,200,$,"[{""alias"": ""cafes"", ""title"": ""Cafes""}, {""alias..."
...,...,...,...,...,...,...,...
187,up & adam eatz,Bo_TnA3HGE12c7vJ91UCBA,"3903 Canal St, New Orleans, LA 70119",4.2,193,$$,"[{""alias"": ""coffee"", ""title"": ""Coffee & Tea""},..."
188,juan's flying burrito,dGeXdSMah56gEHwZNaRQKA,"4724 S Carrollton Ave, New Orleans, LA 70119",3.6,565,$$,"[{""alias"": ""mexican"", ""title"": ""Mexican""}, {""a..."
189,mid-city yacht club,jrGaVL6KIwYpwivse_eKXw,"440 S St Patrick St, New Orleans, LA 70119",4.3,166,$$,"[{""alias"": ""sportsbars"", ""title"": ""Sports Bars..."
190,brocato’s kitchen,uRMqISU-SpXRsx0fiJVhcQ,"422 S Broad St, New Orleans, LA 70119",4.6,9,$$,"[{""alias"": ""seafood"", ""title"": ""Seafood""}, {""a..."


In [83]:
print(combined_df.dtypes)
print(combined_df['Cuisine'].head(50))



Name              object
Business ID       object
Address           object
Rating           float64
Total Ratings      int64
Price             object
Cuisine           object
dtype: object
0                                                  None
1     [{'alias': 'southern', 'title': 'Southern'}, {...
2     [{'alias': 'bars', 'title': 'Bars'}, {'alias':...
3     [{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...
4     [{'alias': 'cafes', 'title': 'Cafes'}, {'alias...
5     [{'alias': 'popuprestaurants', 'title': 'Pop-U...
6     [{'alias': 'japanese', 'title': 'Japanese'}, {...
7     [{'alias': 'breakfast_brunch', 'title': 'Break...
8            [{'alias': 'mexican', 'title': 'Mexican'}]
9     [{'alias': 'soulfood', 'title': 'Soul Food'}, ...
10    [{'alias': 'tacos', 'title': 'Tacos'}, {'alias...
11    [{'alias': 'japanese', 'title': 'Japanese'}, {...
12               [{'alias': 'pizza', 'title': 'Pizza'}]
13    [{'alias': 'bars', 'title': 'Bars'}, {'alias':...
14    [{'alias': 'chinese',

In [91]:
combined_df['Cuisine'] = combined_df['Cuisine'].apply(lambda x: json.dumps(x) if isinstance(x, list) else None)


In [92]:
data_tuples = [
    tuple(None if isinstance(x, (np.float64, np.ndarray)) and np.isnan(x) else x for x in row)
    for row in combined_df.iloc[1:].itertuples(index=False, name=None)  # Skip the first row
]

In [93]:
data_tuples

[('ye olde college inn',
  'yY3kq6_2m1FyY9MgaiHgoQ',
  '3000 S Carrollton Ave, New Orleans, LA 70118',
  4.2,
  272,
  '$$',
  '[{"alias": "southern", "title": "Southern"}, {"alias": "cajun", "title": "Cajun/Creole"}, {"alias": "newamerican", "title": "New American"}]'),
 ('nice guys nola',
  'AZJpX5slpbZUTU9INbwu0g',
  '7910 Earhart Blvd, New Orleans, LA 70125',
  3.2,
  459,
  '$$',
  '[{"alias": "bars", "title": "Bars"}, {"alias": "tradamerican", "title": "American"}, {"alias": "chicken_wings", "title": "Chicken Wings"}]'),
 ('boucherie',
  'rsMbMeW7IbXxPFh4WwbddQ',
  '8115 Jeannette St, New Orleans, LA 70118',
  4.3,
  627,
  '$$',
  '[{"alias": "bbq", "title": "Barbeque"}, {"alias": "southern", "title": "Southern"}]'),
 ('luca eats',
  '854d-ziK_hQFmbNdvF3ZsA',
  '7329 Cohn St, New Orleans, LA 70118',
  4.5,
  200,
  '$',
  '[{"alias": "cafes", "title": "Cafes"}, {"alias": "breakfast_brunch", "title": "Breakfast & Brunch"}, {"alias": "tradamerican", "title": "American"}]'),
 ('que

In [94]:
insert_query = """
    INSERT INTO restaurant_info (name, business_id, address, rating, total_ratings, price, cuisine)
    VALUES (%s, %s, %s, %s, %s, %s, %s);
    """
cursor.executemany(insert_query, data_tuples)
conn.commit()


In [ ]:
def yelp_review_highlights(api_key, business_id):

    url = f"https://api.yelp.com/v3/businesses/{business_id}/review_highlights?count=5"
    headers = {'Authorization': f'Bearer {api_key}'}

    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: Unable to fetch reviews from Yelp API. Status Code: {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame if there's an error
    
    # Parse the JSON response
    reviews = response.json().get('reviews', [])

    # Create a DataFrame from the reviews data
    reviews_df = pd.DataFrame(reviews)

    return reviews_df